*This script generates tables with data to use them in ANOVAs and t-tests further. OUTPUT -- replication_data.csv, main_data.csv, main_data_sumActivatoin.csv, lesion_data.csv, lesion_data_sumActivatoin.csv*

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats

# 1. Generate tables with data for Garagnani & Pulvermüller (2016) replication  

### Create table template

In [2]:
#create column names for the replication table
columns_replic = ['net_num', 'ExtraPeri', 'FrontoTemp', 'Modality', 'area', 'WordType', 'activation']

#order of areas in this table
areas = ['M1-L', 'PM-L','PF-L', 'V1', 'TO', 'AT', 'M1-i', 'PM-i', 'PF-i', 'A1', 'AB', 'PB']

nets_num = 13

In [3]:
replication_data = pd.DataFrame(index=[i for i in range (24*nets_num)], columns=columns_replic)


In [4]:
# fill in values for all columns except activation (as activation will be filled in with CA data)

#levels of var -- net_num = 13 nets (24 values per one net)  
# -> ExtraPeri = 2 lvls(12 values per each lvl) 
# -> FrontoTemp = 2 lvls (6 values per each lvl) -> Modality = 3 lvls (2 values per each lvl) 
# -> WordType = 2 lvls


net_num_list = []
for i in range(nets_num):
    net_num_list += [i]*24

    
ExtraPeri_list = []
for i in range(nets_num*2):
    if i%2 == 0:
        ExtraPeri_list += ['extra']*12
    else:
        ExtraPeri_list += ['peri']*12
        

FrontoTemp_list = []
for i in range(nets_num*2*2):
    if i%2 == 0:
        FrontoTemp_list += ['frontal']*6
    else:
        FrontoTemp_list += ['temporal']*6
        
        
Modality_list = []
for i in range(nets_num*2*2*3):
    if i%3 == 0:
        Modality_list += ['primary']*2
    elif i%3 == 1:
        Modality_list += ['secondary']*2
    else:
        Modality_list += ['central']*2

        
area_list = []
for i in range(nets_num):
    for area in areas:
        area_list += [area]*2
        

WordType_list = []
for i in range(nets_num*2*2*3*2):
    if i%2 == 0:
        WordType_list += ['object']
    else:
        WordType_list += ['action']
        



replication_data['net_num'] = net_num_list   
replication_data['ExtraPeri'] = ExtraPeri_list   
replication_data['FrontoTemp'] = FrontoTemp_list   
replication_data['Modality'] = Modality_list   
replication_data['area'] = area_list   
replication_data['WordType'] = WordType_list      


In [5]:
replication_data.head(8)

net_num ExtraPeri FrontoTemp   Modality  area WordType activation
0        0     extra    frontal    primary  M1-L   object        NaN
1        0     extra    frontal    primary  M1-L   action        NaN
2        0     extra    frontal  secondary  PM-L   object        NaN
3        0     extra    frontal  secondary  PM-L   action        NaN
4        0     extra    frontal    central  PF-L   object        NaN
5        0     extra    frontal    central  PF-L   action        NaN
6        0     extra   temporal    primary    V1   object        NaN
7        0     extra   temporal    primary    V1   action        NaN

### Load CAs and calculate mean activations (averaging object CAs within each net and action CAs within each net)

In [6]:
indexes = ['ca1', 'ca2', 'ca3', 'ca4', 'ca5', 'ca6', 'ca7', 'ca8', 'ca9', 'ca10', 'ca11', 'ca12']

columns = ['V1', 'TO', 'AT', 'PF-L', 'PM-L', 'M1-L','A1', 'AB', 'PB', 'PF-i', 'PM-i', 'M1-i']

nets_num = 13

In [7]:
#get ca-structure wo SD
lines = []
for machine_num in range(nets_num):
    f = open("CA-structure_u"+ str(machine_num) + '_adj'+ ".txt", "r")
    f.readline()
    for line_num in range(12):
        line = [int(cell_num) for cell_num in f.readline().split()[2:]]
        lines.append(line)
    
data_ca = pd.DataFrame(lines, index = indexes*nets_num, columns = columns)

In [8]:
obj_lines = []
for i in range(13):
    line = list(data_ca[12*i:12*i+6].mean())
    obj_lines.append(line)
    
obj_mean_cells_no_ca = pd.DataFrame(obj_lines, columns = columns)

act_lines = []
for i in range(13):
    line = list(data_ca[12*i+6:12*i+12].mean())
    act_lines.append(line)
    
act_mean_cells_no_ca = pd.DataFrame(act_lines, columns = columns)

### Fill in the table with mean activations

In [9]:
# fill in the replication table for OBJECTS
for net_num in range(13):
    for area in areas:
        replication_data['activation'][(replication_data['net_num'] == net_num) &
                                (replication_data['area'] == area) & 
                                (replication_data['WordType'] == 'object')] = obj_mean_cells_no_ca.loc[net_num][area]
                                

# fill in the replication table for ACTIONS
for net_num in range(13):
    for area in areas:
        replication_data['activation'][(replication_data['net_num'] == net_num) &
                                (replication_data['area'] == area) & 
                                (replication_data['WordType'] == 'action')] = act_mean_cells_no_ca.loc[net_num][area]
        

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Generate csv file with data for replication  

In [10]:
replication_data.to_csv('replication_data.csv', index=False)

# 2. Generate tables with data from our current experiment

## 2.1 generate one table with all data 

### Create table template

In [11]:
#create column names for the main table
columns_main = ['net_num', 'ExtraPeri', 'FrontoTemp', 'Modality', 'area', 'WordType', 'SDType', 'Severity', 'activation']

#order of areas in this main table
areas = ['M1-L', 'PM-L','PF-L', 'V1', 'TO', 'AT', 'M1-i', 'PM-i', 'PF-i', 'A1', 'AB', 'PB']

nets_num = 13


In [12]:
main_data = pd.DataFrame(index=[i for i in range (192*nets_num)], columns=columns_main)

In [13]:
# fill in values for all columns except activation (as activation will be filled in with recognition data)

#levels of var -- net_num (192 values per one net) -> ExtraPeri (96 values per each) 
# -> FrontoTemp (48 values per each) -> Modality (16 values per each) 
# -> WordType (8 values per each) > SDType (4 values per each) -> Severity - 4 lvls


net_num_list = []
for i in range(nets_num):
    net_num_list += [i]*192

    
ExtraPeri_list = []
for i in range(nets_num*2):
    if i%2 == 0:
        ExtraPeri_list += ['extra']*96
    else:
        ExtraPeri_list += ['peri']*96
        

FrontoTemp_list = []
for i in range(nets_num*2*2):
    if i%2 == 0:
        FrontoTemp_list += ['frontal']*48
    else:
        FrontoTemp_list += ['temporal']*48
        
        
Modality_list = []
for i in range(nets_num*2*2*3):
    if i%3 == 0:
        Modality_list += ['primary']*16
    elif i%3 == 1:
        Modality_list += ['secondary']*16
    else:
        Modality_list += ['central']*16

        
area_list = []
for i in range(nets_num):
    for area in areas:
        area_list += [area]*16
        

WordType_list = []
for i in range(nets_num*2*2*3*2):
    if i%2 == 0:
        WordType_list += ['object']*8
    else:
        WordType_list += ['action']*8
        

SDType_list = []
for i in range(nets_num*2*2*3*2*2):
    if i%2 == 0:
        SDType_list += ['GM']*4
    else:
        SDType_list += ['WM']*4

        
Severity_list = ['no', '30%', '60%', '90%']*nets_num*2*2*3*2*2




main_data['net_num'] = net_num_list   
main_data['ExtraPeri'] = ExtraPeri_list   
main_data['FrontoTemp'] = FrontoTemp_list   
main_data['Modality'] = Modality_list   
main_data['area'] = area_list   
main_data['WordType'] = WordType_list   
main_data['SDType'] = SDType_list   
main_data['Severity'] = Severity_list   


In [14]:
main_data.head(8)

net_num ExtraPeri FrontoTemp Modality  area WordType SDType Severity  \
0        0     extra    frontal  primary  M1-L   object     GM       no   
1        0     extra    frontal  primary  M1-L   object     GM      30%   
2        0     extra    frontal  primary  M1-L   object     GM      60%   
3        0     extra    frontal  primary  M1-L   object     GM      90%   
4        0     extra    frontal  primary  M1-L   object     WM       no   
5        0     extra    frontal  primary  M1-L   object     WM      30%   
6        0     extra    frontal  primary  M1-L   object     WM      60%   
7        0     extra    frontal  primary  M1-L   object     WM      90%   

  activation  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  
5        NaN  
6        NaN  
7        NaN

### Load CAs and calculate mean activations per net (averaging object CAs within each net and action CAs within each net) for each area and each condition -- WordType x SDType x Severity

In [15]:
indexes = ['ca1', 'ca2', 'ca3', 'ca4', 'ca5', 'ca6', 'ca7', 'ca8', 'ca9', 'ca10', 'ca11', 'ca12']

columns = ['V1', 'TO', 'AT', 'PF-L', 'PM-L', 'M1-L','A1', 'AB', 'PB', 'PF-i', 'PM-i', 'M1-i']

nets_num = 13


**no SD data**

In [16]:
#get recognition wo SD
lines = []
for machine_num in range(nets_num):
    f = open("CA-structure_u"+ str(machine_num) + '_adj_re'+ ".txt", "r")
    f.readline()
    for line_num in range(12):
        line = [int(cell_num) for cell_num in f.readline().split()[2:]]
        lines.append(line)
    
data_re = pd.DataFrame(lines, index = indexes*nets_num, columns = columns)

In [17]:
obj_lines = []
for i in range(13): #!!!!!! here add net_num instead of 13
    line = list(data_re[12*i:12*i+6].mean())
    obj_lines.append(line)
    
obj_mean_cells_no = pd.DataFrame(obj_lines, columns = columns)

act_lines = []
for i in range(13):
    line = list(data_re[12*i+6:12*i+12].mean())
    act_lines.append(line)
    
act_mean_cells_no = pd.DataFrame(act_lines, columns = columns)

**GM SD data**

1. 30% SD

In [18]:
#get recognition for SD GM30
lines = []
for machine_num in range(nets_num):
    f = open("CA-structure_u"+ str(machine_num) +"_gm30_re"+".txt", "r")
    f.readline()
    for line_num in range(12):
        line = [int(cell_num) for cell_num in f.readline().split()[2:]]
        lines.append(line)
    
data_gm30_re = pd.DataFrame(lines, index = indexes*nets_num, columns = columns)

In [19]:
obj_lines = []
for i in range(13):
    line = list(data_gm30_re[12*i:12*i+6].mean())
    obj_lines.append(line)
    
obj_mean_cells_gm30 = pd.DataFrame(obj_lines, columns = columns)


act_lines = []
for i in range(13):
    line = list(data_gm30_re[12*i+6:12*i+12].mean())
    act_lines.append(line)
    
act_mean_cells_gm30 = pd.DataFrame(act_lines, columns = columns)


2. 60% SD

In [20]:
#get recognition for SD GM60
lines = []
for machine_num in range(nets_num):
    f = open("CA-structure_u"+ str(machine_num) +"_gm60_re"+".txt", "r")
    f.readline()
    for line_num in range(12):
        line = [int(cell_num) for cell_num in f.readline().split()[2:]]
        lines.append(line)
    
data_gm60_re = pd.DataFrame(lines, index = indexes*nets_num, columns = columns)

In [21]:
obj_lines = []
for i in range(13):
    line = list(data_gm60_re[12*i:12*i+6].mean())
    obj_lines.append(line)
    
obj_mean_cells_gm60 = pd.DataFrame(obj_lines, columns = columns)


act_lines = []
for i in range(13):
    line = list(data_gm60_re[12*i+6:12*i+12].mean())
    act_lines.append(line)
    
act_mean_cells_gm60 = pd.DataFrame(act_lines, columns = columns)

3. 90% SD

In [22]:
#get recognition for SD GM90
lines = []
for machine_num in range(nets_num):
    f = open("CA-structure_u"+ str(machine_num) +"_gm90_re"+".txt", "r")
    f.readline()
    for line_num in range(12):
        line = [int(cell_num) for cell_num in f.readline().split()[2:]]
        lines.append(line)
    
data_gm90_re = pd.DataFrame(lines, index = indexes*nets_num, columns = columns)

In [23]:
obj_lines = []
for i in range(13):
    line = list(data_gm90_re[12*i:12*i+6].mean())
    obj_lines.append(line)
    
obj_mean_cells_gm90 = pd.DataFrame(obj_lines, columns = columns)


act_lines = []
for i in range(13):
    line = list(data_gm90_re[12*i+6:12*i+12].mean())
    act_lines.append(line)
    
act_mean_cells_gm90 = pd.DataFrame(act_lines, columns = columns)

**WM SD data**

1. 30% SD

In [24]:
#get recognition for SD WM30
lines = []
for machine_num in range(nets_num):
    f = open("CA-structure_u"+ str(machine_num) +"_wm30_re"+".txt", "r")
    f.readline()
    for line_num in range(12):
        line = [int(cell_num) for cell_num in f.readline().split()[2:]]
        lines.append(line)
    
data_wm30_re = pd.DataFrame(lines, index = indexes*nets_num, columns = columns)

In [25]:
obj_lines = []
for i in range(13):
    line = list(data_wm30_re[12*i:12*i+6].mean())
    obj_lines.append(line)
    
obj_mean_cells_wm30 = pd.DataFrame(obj_lines, columns = columns)


act_lines = []
for i in range(13):
    line = list(data_wm30_re[12*i+6:12*i+12].mean())
    act_lines.append(line)
    
act_mean_cells_wm30 = pd.DataFrame(act_lines, columns = columns)


2. 60% SD

In [26]:
#get recognition for SD WM60
lines = []
for machine_num in range(nets_num):
    f = open("CA-structure_u"+ str(machine_num) +"_wm60_re"+".txt", "r")
    f.readline()
    for line_num in range(12):
        line = [int(cell_num) for cell_num in f.readline().split()[2:]]
        lines.append(line)
    
data_wm60_re = pd.DataFrame(lines, index = indexes*nets_num, columns = columns)

In [27]:
obj_lines = []
for i in range(13):
    line = list(data_wm60_re[12*i:12*i+6].mean())
    obj_lines.append(line)
    
obj_mean_cells_wm60 = pd.DataFrame(obj_lines, columns = columns)


act_lines = []
for i in range(13):
    line = list(data_wm60_re[12*i+6:12*i+12].mean())
    act_lines.append(line)
    
act_mean_cells_wm60 = pd.DataFrame(act_lines, columns = columns)


3. 90% SD

In [28]:
#get recognition for SD WM90
lines = []
for machine_num in range(nets_num):
    f = open("CA-structure_u"+ str(machine_num) +"_wm90_re"+".txt", "r")
    f.readline()
    for line_num in range(12):
        line = [int(cell_num) for cell_num in f.readline().split()[2:]]
        lines.append(line)
    
data_wm90_re = pd.DataFrame(lines, index = indexes*nets_num, columns = columns)

In [29]:
obj_lines = []
for i in range(13):
    line = list(data_wm90_re[12*i:12*i+6].mean())
    obj_lines.append(line)
    
obj_mean_cells_wm90 = pd.DataFrame(obj_lines, columns = columns)


act_lines = []
for i in range(13):
    line = list(data_wm90_re[12*i+6:12*i+12].mean())
    act_lines.append(line)
    
act_mean_cells_wm90 = pd.DataFrame(act_lines, columns = columns)


### Fill in the main table with mean activations

In [30]:
# fill in the main table for --- object, GM, no 
for net_num in range(13): #change 13 for nets_num !!!!!!
    for area in areas:
        main_data['activation'][(main_data['net_num'] == net_num) &
                                (main_data['area'] == area) & 
                                (main_data['WordType'] == 'object') & 
                                (main_data['SDType'] == 'GM') &
                                (main_data['Severity'] == 'no')] = obj_mean_cells_no.loc[net_num][area]
       
    

# fill in the main table for --- action, GM, no 
for net_num in range(13):
    for area in areas:
        main_data['activation'][(main_data['net_num'] == net_num) &
                                (main_data['area'] == area) & 
                                (main_data['WordType'] == 'action') & 
                                (main_data['SDType'] == 'GM') &
                                (main_data['Severity'] == 'no')] = act_mean_cells_no.loc[net_num][area]
        
        
# fill in the main table for --- object, WM, no 
for net_num in range(13):
    for area in areas:
        main_data['activation'][(main_data['net_num'] == net_num) &
                                (main_data['area'] == area) & 
                                (main_data['WordType'] == 'object') & 
                                (main_data['SDType'] == 'WM') &
                                (main_data['Severity'] == 'no')] = obj_mean_cells_no.loc[net_num][area]
       
    

# fill in the main table for --- action, WM, no 
for net_num in range(13):
    for area in areas:
        main_data['activation'][(main_data['net_num'] == net_num) &
                                (main_data['area'] == area) & 
                                (main_data['WordType'] == 'action') & 
                                (main_data['SDType'] == 'WM') &
                                (main_data['Severity'] == 'no')] = act_mean_cells_no.loc[net_num][area]
    
#__________________________________________________________
    
    
# fill in the main table for --- object, GM, 30% 
for net_num in range(13):
    for area in areas:
        main_data['activation'][(main_data['net_num'] == net_num) &
                                (main_data['area'] == area) & 
                                (main_data['WordType'] == 'object') & 
                                (main_data['SDType'] == 'GM') &
                                (main_data['Severity'] == '30%')] = obj_mean_cells_gm30.loc[net_num][area]
       
    

# fill in the main table for --- action, GM, 30% 
for net_num in range(13):
    for area in areas:
        main_data['activation'][(main_data['net_num'] == net_num) &
                                (main_data['area'] == area) & 
                                (main_data['WordType'] == 'action') & 
                                (main_data['SDType'] == 'GM') &
                                (main_data['Severity'] == '30%')] = act_mean_cells_gm30.loc[net_num][area]
        
        
# fill in the main table for --- object, WM, 30% 
for net_num in range(13):
    for area in areas:
        main_data['activation'][(main_data['net_num'] == net_num) &
                                (main_data['area'] == area) & 
                                (main_data['WordType'] == 'object') & 
                                (main_data['SDType'] == 'WM') &
                                (main_data['Severity'] == '30%')] = obj_mean_cells_wm30.loc[net_num][area]
       
    

# fill in the main table for --- action, WM, 30% 
for net_num in range(13):
    for area in areas:
        main_data['activation'][(main_data['net_num'] == net_num) &
                                (main_data['area'] == area) & 
                                (main_data['WordType'] == 'action') & 
                                (main_data['SDType'] == 'WM') &
                                (main_data['Severity'] == '30%')] = act_mean_cells_wm30.loc[net_num][area]

        
        
#__________________________________________________________
    
    
# fill in the main table for --- object, GM, 60% 
for net_num in range(13):
    for area in areas:
        main_data['activation'][(main_data['net_num'] == net_num) &
                                (main_data['area'] == area) & 
                                (main_data['WordType'] == 'object') & 
                                (main_data['SDType'] == 'GM') &
                                (main_data['Severity'] == '60%')] = obj_mean_cells_gm60.loc[net_num][area]
       
    

# fill in the main table for --- action, GM, 60% 
for net_num in range(13):
    for area in areas:
        main_data['activation'][(main_data['net_num'] == net_num) &
                                (main_data['area'] == area) & 
                                (main_data['WordType'] == 'action') & 
                                (main_data['SDType'] == 'GM') &
                                (main_data['Severity'] == '60%')] = act_mean_cells_gm60.loc[net_num][area]
        
        
# fill in the main table for --- object, WM, 60% 
for net_num in range(13):
    for area in areas:
        main_data['activation'][(main_data['net_num'] == net_num) &
                                (main_data['area'] == area) & 
                                (main_data['WordType'] == 'object') & 
                                (main_data['SDType'] == 'WM') &
                                (main_data['Severity'] == '60%')] = obj_mean_cells_wm60.loc[net_num][area]
       
    

# fill in the main table for --- action, WM, 60% 
for net_num in range(13):
    for area in areas:
        main_data['activation'][(main_data['net_num'] == net_num) &
                                (main_data['area'] == area) & 
                                (main_data['WordType'] == 'action') & 
                                (main_data['SDType'] == 'WM') &
                                (main_data['Severity'] == '60%')] = act_mean_cells_wm60.loc[net_num][area]
 

    
# ________________________________________________________

# fill in the main table for --- object, GM, 90% 
for net_num in range(13):
    for area in areas:
        main_data['activation'][(main_data['net_num'] == net_num) &
                                (main_data['area'] == area) & 
                                (main_data['WordType'] == 'object') & 
                                (main_data['SDType'] == 'GM') &
                                (main_data['Severity'] == '90%')] = obj_mean_cells_gm90.loc[net_num][area]
       
    

# fill in the main table for --- action, GM, 90% 
for net_num in range(13):
    for area in areas:
        main_data['activation'][(main_data['net_num'] == net_num) &
                                (main_data['area'] == area) & 
                                (main_data['WordType'] == 'action') & 
                                (main_data['SDType'] == 'GM') &
                                (main_data['Severity'] == '90%')] = act_mean_cells_gm90.loc[net_num][area]
        
        
# fill in the main table for --- object, WM, 90% 
for net_num in range(13):
    for area in areas:
        main_data['activation'][(main_data['net_num'] == net_num) &
                                (main_data['area'] == area) & 
                                (main_data['WordType'] == 'object') & 
                                (main_data['SDType'] == 'WM') &
                                (main_data['Severity'] == '90%')] = obj_mean_cells_wm90.loc[net_num][area]
       
    

# fill in the main table for --- action, WM, 90% 
for net_num in range(13):
    for area in areas:
        main_data['activation'][(main_data['net_num'] == net_num) &
                                (main_data['area'] == area) & 
                                (main_data['WordType'] == 'action') & 
                                (main_data['SDType'] == 'WM') &
                                (main_data['Severity'] == '90%')] = act_mean_cells_wm90.loc[net_num][area]
        

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value

### Generate csv file with main table 

In [31]:
main_data.to_csv('main_data.csv', index=False)

## 2.2 Generate table with summed activations in 6 areas either from extrasylvian or from perisylvian

### Create table template


In [32]:
columns_sumActivatoin = ['net_num', 'ExtraPeri', 'WordType', 'SDType', 'Severity', 'activation']

nets_num = 13

main_data_sumActivatoin = pd.DataFrame(index=[i for i in range (32*nets_num)], columns=columns_sumActivatoin)


In [33]:
net_num_list = []
for i in range(nets_num):
    net_num_list += [i]*32

    
ExtraPeri_list = []
for i in range(nets_num*2):
    if i%2 == 0:
        ExtraPeri_list += ['extra']*16
    else:
        ExtraPeri_list += ['peri']*16
        

        

WordType_list = []
for i in range(nets_num*2*2):
    if i%2 == 0:
        WordType_list += ['object']*8
    else:
        WordType_list += ['action']*8
        

SDType_list = []
for i in range(nets_num*2*2*2):
    if i%2 == 0:
        SDType_list += ['GM']*4
    else:
        SDType_list += ['WM']*4

        
Severity_list = ['no', '30%', '60%', '90%']*nets_num*2*2*2




main_data_sumActivatoin['net_num'] = net_num_list   
main_data_sumActivatoin['ExtraPeri'] = ExtraPeri_list   
main_data_sumActivatoin['WordType'] = WordType_list   
main_data_sumActivatoin['SDType'] = SDType_list   
main_data_sumActivatoin['Severity'] = Severity_list 


### Fill in the table with summed activations

In [34]:
for net in range(nets_num):
    for sylvianity in ['extra', 'peri']:
        for word in ['object', 'action']:
            for matter in ['GM', 'WM']:
                for severity in ['no', '30%', '60%', '90%']:
                    main_data_sumActivatoin['activation'][(main_data_sumActivatoin['net_num'] == net) &
                                (main_data_sumActivatoin['ExtraPeri'] == sylvianity) & 
                                (main_data_sumActivatoin['WordType'] == word) & 
                                (main_data_sumActivatoin['SDType'] == matter) &
                                (main_data_sumActivatoin['Severity'] == severity)] = sum(main_data['activation'][(main_data['net_num'] == net) &
                                                                            (main_data['ExtraPeri'] == sylvianity) & 
                                                                            (main_data['WordType'] == word) & 
                                                                            (main_data['SDType'] == matter) &
                                                                            (main_data['Severity'] == severity)])
    

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Generate csv file with main_data_sumActivatoin table 

In [35]:
main_data_sumActivatoin.to_csv('main_data_sumActivatoin.csv', index=False)

# 3. Generate tables with data from nets with lesions

## 3.1 Generate one table with all data

### Create table template


In [36]:
#create column names for the lesion table
columns_lesion = ['net_num', 'ExtraPeri', 'FrontoTemp', 'Modality', 'area', 'WordType', 'LesionType','activation']

#order of areas in this table
areas = ['M1-L', 'PM-L','PF-L', 'V1', 'TO', 'AT', 'M1-i', 'PM-i', 'PF-i', 'A1', 'AB', 'PB']

nets_num = 4

In [37]:
lesion_data = pd.DataFrame(index=[i for i in range (72*nets_num)], columns=columns_lesion)

In [38]:
# fill in values for all columns except activation (as activation will be filled in with CA data)

#levels of var -- net_num (72 values per one net) -> ExtraPeri (36 values per each) 
# -> FrontoTemp (18 values per each) -> Modality (6 values per each) 
# -> WordType (3 values per each) > LesionType -- 3 lvls


net_num_list = []
for i in range(nets_num):
    net_num_list += [i]*72

    
ExtraPeri_list = []
for i in range(nets_num*2):
    if i%2 == 0:
        ExtraPeri_list += ['extra']*36
    else:
        ExtraPeri_list += ['peri']*36
        

FrontoTemp_list = []
for i in range(nets_num*2*2):
    if i%2 == 0:
        FrontoTemp_list += ['frontal']*18
    else:
        FrontoTemp_list += ['temporal']*18
        
        
Modality_list = []
for i in range(nets_num*2*2*3):
    if i%3 == 0:
        Modality_list += ['primary']*6
    elif i%3 == 1:
        Modality_list += ['secondary']*6
    else:
        Modality_list += ['central']*6

        
area_list = []
for i in range(nets_num):
    for area in areas:
        area_list += [area]*6
        

WordType_list = []
for i in range(nets_num*2*2*3*2):
    if i%2 == 0:
        WordType_list += ['object']*3
    else:
        WordType_list += ['action']*3
        

        
LesionType_list = ['no', 'V1', 'M1']*nets_num*2*2*3*2




lesion_data['net_num'] = net_num_list   
lesion_data['ExtraPeri'] = ExtraPeri_list   
lesion_data['FrontoTemp'] = FrontoTemp_list   
lesion_data['Modality'] = Modality_list   
lesion_data['area'] = area_list   
lesion_data['WordType'] = WordType_list   
lesion_data['LesionType'] = LesionType_list   


In [39]:
lesion_data.head(8)

net_num ExtraPeri FrontoTemp   Modality  area WordType LesionType  \
0        0     extra    frontal    primary  M1-L   object         no   
1        0     extra    frontal    primary  M1-L   object         V1   
2        0     extra    frontal    primary  M1-L   object         M1   
3        0     extra    frontal    primary  M1-L   action         no   
4        0     extra    frontal    primary  M1-L   action         V1   
5        0     extra    frontal    primary  M1-L   action         M1   
6        0     extra    frontal  secondary  PM-L   object         no   
7        0     extra    frontal  secondary  PM-L   object         V1   

  activation  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  
5        NaN  
6        NaN  
7        NaN

### Load CAs and calculate mean activations per net (averaging object CAs within each net and action CAs within each net) for each area and each condition -- WordType x LesionType

In [40]:
indexes = ['ca1', 'ca2', 'ca3', 'ca4', 'ca5', 'ca6', 'ca7', 'ca8', 'ca9', 'ca10', 'ca11', 'ca12']

columns = ['V1', 'TO', 'AT', 'PF-L', 'PM-L', 'M1-L','A1', 'AB', 'PB', 'PF-i', 'PM-i', 'M1-i']

nets_num = 4

In [42]:
#load CA without lesions
lines = []
for machine_num in range(nets_num):
    f = open("CA-structure_u"+ str(machine_num) + '_adj_lesion.txt', "r")
    f.readline()
    for line_num in range(12):
        line = [int(cell_num) for cell_num in f.readline().split()[2:]]
        lines.append(line)
    
data_no_lesion = pd.DataFrame(lines, index = indexes*nets_num, columns = columns)

In [43]:
obj_lines = []
for i in range(nets_num): 
    line = list(data_no_lesion[12*i:12*i+6].mean())
    obj_lines.append(line)
    
obj_mean_cells_no_lesion = pd.DataFrame(obj_lines, columns = columns)

act_lines = []
for i in range(nets_num):
    line = list(data_no_lesion[12*i+6:12*i+12].mean())
    act_lines.append(line)
    
act_mean_cells_no_lesion = pd.DataFrame(act_lines, columns = columns)

In [44]:
#load CA with V1 lesions
lines = []
for machine_num in range(nets_num):
    f = open("CA-structure_u"+ str(machine_num) + '_lV1'+ ".txt", "r")
    f.readline()
    for line_num in range(12):
        line = [int(cell_num) for cell_num in f.readline().split()[2:]]
        lines.append(line)
    
data_V1_lesion = pd.DataFrame(lines, index = indexes*nets_num, columns = columns)

In [45]:
obj_lines = []
for i in range(nets_num): 
    line = list(data_V1_lesion[12*i:12*i+6].mean())
    obj_lines.append(line)
    
obj_mean_cells_V1_lesion = pd.DataFrame(obj_lines, columns = columns)

act_lines = []
for i in range(nets_num):
    line = list(data_V1_lesion[12*i+6:12*i+12].mean())
    act_lines.append(line)
    
act_mean_cells_V1_lesion = pd.DataFrame(act_lines, columns = columns)

In [46]:
#load CA with M1 lesions
lines = []
for machine_num in range(nets_num):
    f = open("CA-structure_u"+ str(machine_num) + '_lM1'+ ".txt", "r")
    f.readline()
    for line_num in range(12):
        line = [int(cell_num) for cell_num in f.readline().split()[2:]]
        lines.append(line)
    
data_M1_lesion = pd.DataFrame(lines, index = indexes*nets_num, columns = columns)

In [47]:
obj_lines = []
for i in range(nets_num): 
    line = list(data_M1_lesion[12*i:12*i+6].mean())
    obj_lines.append(line)
    
obj_mean_cells_M1_lesion = pd.DataFrame(obj_lines, columns = columns)

act_lines = []
for i in range(nets_num):
    line = list(data_M1_lesion[12*i+6:12*i+12].mean())
    act_lines.append(line)
    
act_mean_cells_M1_lesion = pd.DataFrame(act_lines, columns = columns)

### Fill in the table with mean activations

In [48]:
# fill in the lesion table for --- object, no lesion 
for net_num in range(nets_num):
    for area in areas:
        lesion_data['activation'][(lesion_data['net_num'] == net_num) &
                                (lesion_data['area'] == area) & 
                                (lesion_data['WordType'] == 'object') & 
                                (lesion_data['LesionType'] == 'no')] = obj_mean_cells_no_lesion.loc[net_num][area]
        
        

# fill in the lesion table for --- object, V1 lesion 
for net_num in range(nets_num):
    for area in areas:
        lesion_data['activation'][(lesion_data['net_num'] == net_num) &
                                (lesion_data['area'] == area) & 
                                (lesion_data['WordType'] == 'object') & 
                                (lesion_data['LesionType'] == 'V1')] = obj_mean_cells_V1_lesion.loc[net_num][area]
        
        
        
# fill in the lesion table for --- object, M1 lesion 
for net_num in range(nets_num):
    for area in areas:
        lesion_data['activation'][(lesion_data['net_num'] == net_num) &
                                (lesion_data['area'] == area) & 
                                (lesion_data['WordType'] == 'object') & 
                                (lesion_data['LesionType'] == 'M1')] = obj_mean_cells_M1_lesion.loc[net_num][area]
        
        
# ---------------------------------

# fill in the lesion table for --- action, no lesion 
for net_num in range(nets_num):
    for area in areas:
        lesion_data['activation'][(lesion_data['net_num'] == net_num) &
                                (lesion_data['area'] == area) & 
                                (lesion_data['WordType'] == 'action') & 
                                (lesion_data['LesionType'] == 'no')] = act_mean_cells_no_lesion.loc[net_num][area]
        
        

# fill in the lesion table for --- action, V1 lesion 
for net_num in range(nets_num):
    for area in areas:
        lesion_data['activation'][(lesion_data['net_num'] == net_num) &
                                (lesion_data['area'] == area) & 
                                (lesion_data['WordType'] == 'action') & 
                                (lesion_data['LesionType'] == 'V1')] = act_mean_cells_V1_lesion.loc[net_num][area]
        
        
        
# fill in the lesion table for --- action, M1 lesion 
for net_num in range(nets_num):
    for area in areas:
        lesion_data['activation'][(lesion_data['net_num'] == net_num) &
                                (lesion_data['area'] == area) & 
                                (lesion_data['WordType'] == 'action') & 
                                (lesion_data['LesionType'] == 'M1')] = act_mean_cells_M1_lesion.loc[net_num][area]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning

### Generate csv file with the lesion table 

In [49]:
lesion_data.to_csv('lesion_data.csv', index=False)

## 3.2 Generate table with summed activations in 6 areas either from extrasylvian or from perisylvian

### Create table template


In [50]:
columns_lesion_sumActivatoin = ['net_num', 'ExtraPeri', 'WordType', 'LesionType', 'activation']

nets_num = 4

lesion_data_sumActivatoin = pd.DataFrame(index=[i for i in range (12*nets_num)], columns=columns_lesion_sumActivatoin)


In [51]:
net_num_list = []
for i in range(nets_num):
    net_num_list += [i]*12

    
ExtraPeri_list = []
for i in range(nets_num*2):
    if i%2 == 0:
        ExtraPeri_list += ['extra']*6
    else:
        ExtraPeri_list += ['peri']*6
        


WordType_list = []
for i in range(nets_num*2*2):
    if i%2 == 0:
        WordType_list += ['object']*3
    else:
        WordType_list += ['action']*3
        

        
LesionType_list = ['no', 'V1', 'M1']*nets_num*2*2



lesion_data_sumActivatoin['net_num'] = net_num_list   
lesion_data_sumActivatoin['ExtraPeri'] = ExtraPeri_list   
lesion_data_sumActivatoin['WordType'] = WordType_list   
lesion_data_sumActivatoin['LesionType'] = LesionType_list   


### Fill in the table with summed activations


In [52]:
for net in range(nets_num):
    for sylvianity in ['extra', 'peri']:
        for word in ['object', 'action']:
            for lesion in ['no', 'V1', 'M1']:
                lesion_data_sumActivatoin['activation'][(lesion_data_sumActivatoin['net_num'] == net) &
                            (lesion_data_sumActivatoin['ExtraPeri'] == sylvianity) & 
                            (lesion_data_sumActivatoin['WordType'] == word) & 
                            (lesion_data_sumActivatoin['LesionType'] == lesion)] = sum(lesion_data['activation'][(lesion_data['net_num'] == net) &
                                                                        (lesion_data['ExtraPeri'] == sylvianity) & 
                                                                        (lesion_data['WordType'] == word) & 
                                                                        (lesion_data['LesionType'] == lesion)])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


### Generate csv file 

In [53]:
lesion_data_sumActivatoin.to_csv('lesion_data_sumActivatoin.csv', index=False)